[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Mario-Carmona/SARA_Chatbot/Modelo/inference_gpt_j_6B_8bit.ipynb)

# **Modelo**

## **Dependencias e Imports**

In [ ]:
%pip install transformers==4.14.1
%pip install bitsandbytes-cuda111==0.26.0
%pip install datasets==1.16.1

import transformers

import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import custom_fwd, custom_bwd

from bitsandbytes.functional import quantize_blockwise, dequantize_blockwise

from tqdm.auto import tqdm

## **Convertir modelo a 8 bits**

In [ ]:
class FrozenBNBLinear(nn.Module):
    def __init__(self, weight, absmax, code, bias=None):
        assert isinstance(bias, nn.Parameter) or bias is None
        super().__init__()
        self.out_features, self.in_features = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
        self.bias = bias
 
    def forward(self, input):
        output = DequantizeAndLinear.apply(input, self.weight, self.absmax, self.code, self.bias)
        if self.adapter:
            output += self.adapter(input)
        return output
 
    @classmethod
    def from_linear(cls, linear: nn.Linear) -> "FrozenBNBLinear":
        weights_int8, state = quantize_blockise_lowmemory(linear.weight)
        return cls(weights_int8, *state, linear.bias)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.in_features}, {self.out_features})"
 
 
class DequantizeAndLinear(torch.autograd.Function): 
    @staticmethod
    @custom_fwd
    def forward(ctx, input: torch.Tensor, weights_quantized: torch.ByteTensor,
                absmax: torch.FloatTensor, code: torch.FloatTensor, bias: torch.FloatTensor):
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        ctx.save_for_backward(input, weights_quantized, absmax, code)
        ctx._has_bias = bias is not None
        return F.linear(input, weights_deq, bias)
 
    @staticmethod
    @custom_bwd
    def backward(ctx, grad_output: torch.Tensor):
        assert not ctx.needs_input_grad[1] and not ctx.needs_input_grad[2] and not ctx.needs_input_grad[3]
        input, weights_quantized, absmax, code = ctx.saved_tensors
        # grad_output: [*batch, out_features]
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        grad_input = grad_output @ weights_deq
        grad_bias = grad_output.flatten(0, -2).sum(dim=0) if ctx._has_bias else None
        return grad_input, None, None, None, grad_bias
 
 
class FrozenBNBEmbedding(nn.Module):
    def __init__(self, weight, absmax, code):
        super().__init__()
        self.num_embeddings, self.embedding_dim = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
 
    def forward(self, input, **kwargs):
        with torch.no_grad():
            # note: both quantuized weights and input indices are *not* differentiable
            weight_deq = dequantize_blockwise(self.weight, absmax=self.absmax, code=self.code)
            output = F.embedding(input, weight_deq, **kwargs)
        if self.adapter:
            output += self.adapter(input)
        return output 
 
    @classmethod
    def from_embedding(cls, embedding: nn.Embedding) -> "FrozenBNBEmbedding":
        weights_int8, state = quantize_blockise_lowmemory(embedding.weight)
        return cls(weights_int8, *state)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.num_embeddings}, {self.embedding_dim})"
 
 
def quantize_blockise_lowmemory(matrix: torch.Tensor, chunk_size: int = 2 ** 20):
    assert chunk_size % 4096 == 0
    code = None
    chunks = []
    absmaxes = []
    flat_tensor = matrix.view(-1)
    for i in range((matrix.numel() - 1) // chunk_size + 1):
        input_chunk = flat_tensor[i * chunk_size: (i + 1) * chunk_size].clone()
        quantized_chunk, (absmax_chunk, code) = quantize_blockwise(input_chunk, code=code)
        chunks.append(quantized_chunk)
        absmaxes.append(absmax_chunk)
 
    matrix_i8 = torch.cat(chunks).reshape_as(matrix)
    absmax = torch.cat(absmaxes)
    return matrix_i8, (absmax, code)
 
 
def convert_to_int8(model):
    """Convert linear and embedding modules to 8-bit with optional adapters"""
    for module in list(model.modules()):
        for name, child in module.named_children():
            if isinstance(child, nn.Linear):
                print(name, child)
                setattr( 
                    module,
                    name,
                    FrozenBNBLinear(
                        weight=torch.zeros(child.out_features, child.in_features, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                        bias=child.bias,
                    ),
                )
            elif isinstance(child, nn.Embedding):
                setattr(
                    module,
                    name,
                    FrozenBNBEmbedding(
                        weight=torch.zeros(child.num_embeddings, child.embedding_dim, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                    )
                )

In [ ]:
class GPTJBlock(transformers.models.gptj.modeling_gptj.GPTJBlock):
    def __init__(self, config):
        super().__init__(config)

        convert_to_int8(self.attn)
        convert_to_int8(self.mlp)


class GPTJModel(transformers.models.gptj.modeling_gptj.GPTJModel):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)
        

class GPTJForCausalLM(transformers.models.gptj.modeling_gptj.GPTJForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)


transformers.models.gptj.modeling_gptj.GPTJBlock = GPTJBlock  # monkey-patch GPT-J

## **Definición de la clase Model**

In [ ]:
class Model:
  def __init__(self, pretrained_model_name, tokenizer_model_name):
    self.pretrained_model_name = pretrained_model_name
    self.tokenizer_model_name = tokenizer_model_name

    self.mount_model()

  def mount_model(self):
    self.config = transformers.GPTJConfig.from_pretrained(self.tokenizer_model_name)
    self.tokenizer = transformers.AutoTokenizer.from_pretrained(self.tokenizer_model_name)

    self.gpt = GPTJForCausalLM.from_pretrained(self.pretrained_model_name, low_cpu_mem_usage=True)

    self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    self.gpt.to(self.device)

  def inference(self, question):
    prompt = self.tokenizer(question, return_tensors='pt')
    prompt = {key: value.to(self.device) for key, value in prompt.items()}
    out = self.gpt.generate(**prompt, min_length=128, max_length=128, do_sample=True)
    return self.tokenizer.decode(out[0])

# **Servidor**

## **Dependencias e Imports**

In [ ]:
%pip install flask
%pip install flask-ngrok
%pip install pyngrok==4.1.1

from werkzeug.wrappers import Request
from flask import Flask, request
from flask_ngrok import run_with_ngrok 

from pyngrok import ngrok

## **Definición de la clase Server**

In [ ]:
class Server:
  app = Flask(__name__) 

  def __init__(self, model):
    self.model = model

    self.mount_server()

  def mount_server(self):
    ngrok.set_auth_token("25Pt1y3GT9JEnLudmofnYx6xvrS_2D2gZBpDpQGLhipQSfiHN")
    
    run_with_ngrok(Server.app)

    @Server.app.route("/") 
    def home(): 
      return "Servidor que ejecuta el modelo de GPT-J"

    @Server.app.route("/inferencia", methods=["POST"])
    def inferencia():
      json = request.get_json()
      question = json["question"]

      response = self.model.inference(question)

      print(response)

      return response

    @Server.app.route("/stop", methods=["GET", "POST"])
    def stop():
      print("Stopping server...")
      func = request.environ.get('werkzeug.server.shutdown')
      func()
      print("Server stopped")

      return "Server stopped"

  def run(self):
    Server.app.run()

### Fine-tuning 6-Billion GPT-J in colab with LoRA and 8-bit compression

This notebook is a proof of concept for fine-tuning [GPT-J-6B](https://huggingface.co/EleutherAI/gpt-j-6B) with limited memory. A detailed explanation of how it works can be found in [this model card](https://huggingface.co/hivemind/gpt-j-6B-8bit).

### Converting the model to 8 bits.

We convert EleutherAI's GPT-J-6B model to 8 bits using facebook's [bitsandbytes](https://github.com/facebookresearch/bitsandbytes) library. This reduces the model's size from 20Gb down to just 6Gb.

Note that we don't convert linear layer biases to 8 bit as they take up less that 1% of the model's weight anyway.

# **Configuración servidor**

In [ ]:
#@markdown Seleccionar modelo:

map = {
    "GPT-J-6B-8bit (Modelo base)": {
        "tokenizer": "EleutherAI/gpt-j-6B",
        "model": "hivemind/gpt-j-6B-8bit"
    }
}

model_type = 'GPT-J-6B-8bit (Modelo base)' #@param ["GPT-J-6B-8bit (Modelo base)"]


pretrained_model_name = map["GPT-J-6B-8bit (Modelo base)"]["model"]
tokenizer_model_name = map["GPT-J-6B-8bit (Modelo base)"]["tokenizer"]

model = Model(pretrained_model_name, tokenizer_model_name)

server = Server(model)

######

"""
import ipywidgets as widgets

button = widgets.Button(description="Stop server")
output = widgets.Output()
display(button, output)

def on_button_clicked(b):
  with output:
    print("Stopping server...")
    server.stop()

button.on_click(on_button_clicked)
"""

######


server.run()